### Условие задачи:
$$y'' - x\sqrt{y} = 0, \quad y(0)=0, \quad y(1)=2$$

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import bisect

# Параметры 
x_end = 1.0
y_end = 2.0
h = 0.02
N = int(x_end / h)
x = np.linspace(0, x_end, N+1)

# Метод стрельбы
def solve_shooting(lamb, h, N):
    y = np.zeros(N+1)
    y[0] = 0
    y[1] = y[0] + lamb * h
    
    for k in range(1, N):
        x_k = k * h
        y[k+1] = 2*y[k] - y[k-1] + h**2 * x_k * np.sqrt(abs(y[k]))
    return y

def shooting_error(lamb):
    y = solve_shooting(lamb, h, N)
    return y[-1] - y_end

# Итерационный процесс подбора λ
plt.figure(figsize=(12, 6))
lambdas = [10, 0, 5.0, 2.5, 1.25, 1.875, 1.5625, 1.71875, 1.796875, 1.8359375]
colors = plt.cm.viridis(np.linspace(0, 1, len(lambdas)))

for i, (lamb, color) in enumerate(zip(lambdas, colors)):
    y = solve_shooting(lamb, h, N)
    plt.plot(x, y, '--', color=color, linewidth=1, 
             label=f'λ_{i} = {lamb:.5f}' if i < 6 else None)

# Находим точное значение λ
lamb_final = bisect(shooting_error, 1.7, 1.9, xtol=1e-6)
y_shoot = solve_shooting(lamb_final, h, N)
plt.plot(x, y_shoot, 'b-', linewidth=2, label=f'Финальное решение (λ = {lamb_final:.5f})')

plt.scatter([0, x_end], [0, y_end], c='r', zorder=5, label='Граничные условия')
plt.xlabel('x')
plt.ylabel('y(x)')
plt.title('Метод стрельбы: итерационный подбор λ')
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
# Квазилинеаризация Ньютона
def solve_newton(y_prev, h, N):
    A = np.zeros((N+1, N+1))
    b = np.zeros(N+1)
    
    # Граничные условия
    A[0, 0] = 1
    b[0] = 0
    
    A[N, N] = 1
    b[N] = y_end
    
    # Внутренние точки
    for k in range(1, N):
        x_k = k * h
        y_k = max(y_prev[k], 1e-6)
        
        A[k, k-1] = 1/h**2
        A[k, k] = -2/h**2 - x_k/(2*np.sqrt(y_k))
        A[k, k+1] = 1/h**2
        
        b[k] = x_k * (np.sqrt(y_k) - y_k/(2*np.sqrt(y_k)))
    
    return np.linalg.solve(A, b)

# Итерационный процесс
plt.figure(figsize=(12, 6))
y_newton = np.linspace(0, y_end, N+1)

for i in range(10):
    y_next = solve_newton(y_newton, h, N)
    plt.plot(x, y_next, '--', linewidth=1, 
             label=f'Итерация {i}' if i < 5 else None)
    
    if np.max(np.abs(y_next - y_newton)) < 1e-6:
        y_newton = y_next
        break
    y_newton = y_next

plt.plot(x, y_newton, 'g-', linewidth=2, label='Финальное решение')
plt.scatter([0, x_end], [0, y_end], c='r', zorder=5, label='Граничные условия')
plt.xlabel('x')
plt.ylabel('y(x)')
plt.title('Метод квазилинеаризации Ньютона')
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
# Сравнение методов
plt.figure(figsize=(10, 6))
plt.plot(x, y_shoot, 'b-', linewidth=2, label=f'Метод стрельбы (λ={lamb_final:.5f})')
plt.plot(x, y_newton, 'g--', linewidth=2, label='Метод Ньютона')
plt.scatter([0, x_end], [0, y_end], c='r', zorder=5, label='Граничные условия')
plt.xlabel('x')
plt.ylabel('y(x)')
plt.title('Сравнение методов')
plt.legend()
plt.grid(True)

# Вывод точности
max_diff = np.max(np.abs(y_shoot - y_newton))
print(f"Максимальное расхождение между методами: {max_diff:.6f}")
print(f"Значение на правом конце (стрельба): {y_shoot[-1]:.6f}")
print(f"Значение на правом конце (Ньютон): {y_newton[-1]:.6f}")
plt.show()